In [1]:
%pwd

'/workspaces/chicken-disease-classification/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/workspaces/chicken-disease-classification'

## Entities 

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


## configurtions manager in src/config

In [6]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        print("Configuration Manager Initiated")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        
        create_directories([Path(self.config.prepare_callbacks.root_dir),
                            Path(self.config.prepare_callbacks.tensorboard_root_log_dir),
                            Path(self.config.prepare_callbacks.checkpoint_model_filepath)])
        return PrepareCallbacksConfig(
            root_dir = Path(self.config.prepare_callbacks.root_dir),
            tensorboard_root_log_dir = Path(self.config.prepare_callbacks.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(self.config.prepare_callbacks.checkpoint_model_filepath)
        )
            

## Components

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf # to call VGG16 from keras
import time

In [10]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    
    

## Pipeline

In [17]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks
except Exception as e:
    raise e


Configuration Manager Initiated
[ 2023-09-18 12:55:29,439: INFO: common:  yaml file: config/config.yaml loaded successfully]
[ 2023-09-18 12:55:29,442: INFO: common:  yaml file: params.yaml loaded successfully]
[ 2023-09-18 12:55:29,442: INFO: common:  directory: artifacts created successfully]
[ 2023-09-18 12:55:29,443: INFO: common:  directory: artifacts created successfully]
[ 2023-09-18 12:55:29,444: INFO: common:  directory: artifacts/prepare_callbacks created successfully]
[ 2023-09-18 12:55:29,445: INFO: common:  directory: artifacts/prepare_callbacks created successfully]
[ 2023-09-18 12:55:29,446: INFO: common:  directory: artifacts/prepare_callbacks/tensorboard_log_dir created successfully]
[ 2023-09-18 12:55:29,446: INFO: common:  directory: artifacts/prepare_callbacks/tensorboard_log_dir created successfully]
[ 2023-09-18 12:55:29,447: INFO: common:  directory: artifacts/prepare_callbacks/checkpoint_dir/model.h5 created successfully]
[ 2023-09-18 12:55:29,447: INFO: common:

In [18]:
import time
timestamp  = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-09-18-12-55-36'